<img src="images/spark.png" alt="drawing" width="200"/>

# Introduction Spark DataFrame

A DataFrame is a two-dimensional labeled data structure with columns of potentially different types. You can think of a DataFrame like a spreadsheet, a SQL table, or a dictionary of series objects. Apache Spark DataFrames provide a rich set of functions (select columns, filter, join, aggregate) that allow you to solve common data analysis problems efficiently.

Apache Spark DataFrames are an abstraction built on top of Resilient Distributed Datasets (RDDs). Spark DataFrames and Spark SQL use a unified planning and optimization engine, allowing you to get nearly identical performance across all supported languages (Python, SQL, Scala, and R).




In [1]:
import pyspark
from pyspark.sql import SparkSession

# create a spark session
spark = SparkSession.builder.master("local[1]").appName('SparkDataFrame').getOrCreate()

# get a spark contex
sc = spark.sparkContext


print("APP Name :"+sc.appName);
print("Master   :"+ sc.master);


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/10 21:49:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
APP Name :SparkDataFrame
Master   :local[1]


## Create a DataFrame from RDD

Most Apache Spark queries return a DataFrame. This includes reading from a table, loading data from files, and operations that transform data.



In [2]:


iphones_RDD = sc.parallelize([ ("XS", 2018, 5.65, 2.79, 6.24), \
("XR", 2018, 5.94, 2.98, 6.84),\
("X10", 2017, 5.65, 2.79, 6.13),\
("8Plus", 2017, 6.23, 3.07, 7.12)\
])

names = ['Model', 'Year', 'Height', 'Width', 'Weight']

iphones_df = spark.createDataFrame(iphones_RDD, schema=names)
iphones_df.show()




+-----+----+------+-----+------+
|Model|Year|Height|Width|Weight|
+-----+----+------+-----+------+
|   XS|2018|  5.65| 2.79|  6.24|
|   XR|2018|  5.94| 2.98|  6.84|
|  X10|2017|  5.65| 2.79|  6.13|
|8Plus|2017|  6.23| 3.07|  7.12|
+-----+----+------+-----+------+



## Create a DataFrame from pandas

You can also create a Spark DataFrame from a list or a pandas DataFrame, such as in the following example:

In [3]:
import pandas as pd

data = [[1, "Elia"], [2, "Teo"], [3, "Fang"]]

pdf = pd.DataFrame(data, columns=["i", "n"])

df1 = spark.createDataFrame(pdf)
df1.show()

df2 = spark.createDataFrame(data, schema="id LONG, name STRING")
df2.show()

+---+----+
|  i|   n|
+---+----+
|  1|Elia|
|  2| Teo|
|  3|Fang|
+---+----+

+---+----+
| id|name|
+---+----+
|  1|Elia|
|  2| Teo|
|  3|Fang|
+---+----+



## Load data into a DataFrame from files

You can load data from many supported file formats. The following example uses a dataset available in the datasets directory.

In [4]:
# CSV format
df_csv = spark.read.csv("files/ratings.csv", header=True, inferSchema=True)

df_csv.tail(3)

[Row(userId=5987, movieId=70, rating=5.0, timestamp=1382563110),
 Row(userId=5987, movieId=150, rating=2.5, timestamp=1382564911),
 Row(userId=5987, movieId=152, rating=5.0, timestamp=1382756733)]

Load data from a json file

In [5]:

ratingDF = spark.read.json("files/ratings.json")

# The inferred schema can be visualized using the printSchema() method
ratingDF.printSchema()

ratingDF.show(5)

root
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- userId: long (nullable = true)

+-------+------+----------+------+
|movieId|rating| timestamp|userId|
+-------+------+----------+------+
|    296|   5.0|1147880044|     1|
|    306|   3.5|1147868817|     1|
|    307|   5.0|1147868828|     1|
|    665|   5.0|1147878820|     1|
|    899|   3.5|1147868510|     1|
+-------+------+----------+------+
only showing top 5 rows



 ## DataFrame Operations
 
 Like RDD Spark suporte  support two types of Data Frame operations: transformations and actions
 
* Transformations  create a new dataset from an existing one.
* Actions, which return a value to the driver program after running a computation on the dataset. 

For example, map is a transformation that passes each dataset element through a function and returns a new RDD representing the results. On the other hand, reduce is an action that aggregates all the elements of the RDD using some function and returns the final result to the driver program (although there is also a parallel reduceByKey that returns a distributed dataset).

All transformations in Spark are lazy, in that they do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result to be returned to the driver program. This design enables Spark to run more efficiently. For example, we can realize that a dataset created through map will be used in a reduce and return only the result of the reduce to the driver, rather than the larger mapped dataset.

<img src="images/lazyTranformation.png" alt="drawing" width="900"/>

Here, first an RDD is calculated by reading data from a stable storage and two of the transformations are performed on the RDD and then finally an action is performed to get the result.

By default, each transformed RDD may be recomputed each time you run an action on it. However, you may also persist an RDD in memory using the persist (or cache) method, in which case Spark will keep the elements around on the cluster for much faster access the next time you query it. There is also support for persisting RDDs on disk, or replicated across multiple nodes.

 ### Transformations

#### select( ) - Transformation subsets the columns in the DataFrame


In [6]:
ratingDF = spark.read.json("files/ratings.json")

rating_value_df = ratingDF.select("rating")

rating_value_df.show(5)

+------+
|rating|
+------+
|   5.0|
|   3.5|
|   5.0|
|   5.0|
|   3.5|
+------+
only showing top 5 rows



#### filter( ) transformation filters out the rows based on a condition

In [7]:
ratingDF = spark.read.json("files/ratings.json")

new_df_rating_3 = ratingDF.filter(ratingDF.rating > 3.0)

new_df_rating_3.show(5)

+-------+------+----------+------+
|movieId|rating| timestamp|userId|
+-------+------+----------+------+
|    296|   5.0|1147880044|     1|
|    306|   3.5|1147868817|     1|
|    307|   5.0|1147868828|     1|
|    665|   5.0|1147878820|     1|
|    899|   3.5|1147868510|     1|
+-------+------+----------+------+
only showing top 5 rows



#### groupby() can be used to group a variable

In [9]:
ratingDF = spark.read.json("files/ratings.json")

new_df_groupByRating = ratingDF.groupby('rating')
new_df_groupByRating.count().show()

+------+------+
|rating| count|
+------+------+
|   3.5|110991|
|   4.5| 75423|
|   2.5| 44690|
|   1.0| 26796|
|   4.0|237958|
|   0.5| 12396|
|   3.0|176963|
|   2.0| 59568|
|   1.5| 14815|
|   5.0|127548|
+------+------+



In [12]:
RDD = sc.parallelize(["hello word", "How are you"])
RDD_flatMap = RDD.flatMap(lambda x : x.split(" "))
print ("RDD_flatMap: ", RDD_flatMap.collect()) # action convert to a  List

RDD_flatMap:  ['hello', 'word', 'How', 'are', 'you']


#### orderBy() operation sorts the DataFrame based on one or more columns

In [11]:
ratingDF = spark.read.json("files/ratings.json")

new_df_groupByRating = ratingDF.groupby('rating')
new_df_groupByRating.count().orderBy('rating').show()

+------+------+
|rating| count|
+------+------+
|   0.5| 12396|
|   1.0| 26796|
|   1.5| 14815|
|   2.0| 59568|
|   2.5| 44690|
|   3.0|176963|
|   3.5|110991|
|   4.0|237958|
|   4.5| 75423|
|   5.0|127548|
+------+------+



#### dropDuplicates() removes the duplicate rows of a DataFrame

In [18]:
ratingDF = spark.read.json("files/ratings.json")

newDf = ratingDF.select('userId', 'rating').dropDuplicates()
newDf.count()





39130

#### withColumnRenamed() renames a column in the DataFrame

In [17]:
ratingDF = spark.read.json("files/ratings.json")

newDf = ratingDF.withColumnRenamed('userId','user')
newDf.show()

+-------+------+----------+----+
|movieId|rating| timestamp|user|
+-------+------+----------+----+
|    296|   5.0|1147880044|   1|
|    306|   3.5|1147868817|   1|
|    307|   5.0|1147868828|   1|
|    665|   5.0|1147878820|   1|
|    899|   3.5|1147868510|   1|
|   1088|   4.0|1147868495|   1|
|   1175|   3.5|1147868826|   1|
|   1217|   3.5|1147878326|   1|
|   1237|   5.0|1147868839|   1|
|   1250|   4.0|1147868414|   1|
|   1260|   3.5|1147877857|   1|
|   1653|   4.0|1147868097|   1|
|   2011|   2.5|1147868079|   1|
|   2012|   2.5|1147868068|   1|
|   2068|   2.5|1147869044|   1|
|   2161|   3.5|1147868609|   1|
|   2351|   4.5|1147877957|   1|
|   2573|   4.0|1147878923|   1|
|   2632|   5.0|1147878248|   1|
|   2692|   5.0|1147869100|   1|
+-------+------+----------+----+
only showing top 20 rows



 ### Actions

#### printSchema() method prints the types of columns in the DataFrame

In [20]:
ratingDF = spark.read.json("files/ratings.json")
ratingDF.printSchema()

root
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- userId: long (nullable = true)



#### columns()  prints the columns of a DataFrame

In [22]:
ratingDF = spark.read.json("files/ratings.json")
ratingDF.columns

['movieId', 'rating', 'timestamp', 'userId']

#### describe() operation compute summary statistics of numerical columns in the DataFrame

In [23]:
ratingDF = spark.read.json("files/ratings.json")
ratingDF.describe().show()

+-------+------------------+------------------+--------------------+------------------+
|summary|           movieId|            rating|           timestamp|            userId|
+-------+------------------+------------------+--------------------+------------------+
|  count|            887148|            887148|              887148|            887148|
|   mean|21212.241609066357| 3.533131450445698|1.2092062081188483E9|3041.6676157754964|
| stddev|39197.518981108624|1.0521983681122566| 2.302547044992259E8|1714.4553106441933|
|    min|                 1|               0.5|           789652009|                 1|
|    max|            208939|               5.0|          1574288328|              5987|
+-------+------------------+------------------+--------------------+------------------+

